## Кривоногов Н.В., NLP, практическое задание № 7

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [1]:
#!pip install xlrd

In [2]:
!pip install pymorphy2

In [3]:
!pip install stop_words

In [4]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
#from keras.objectives import categorical_crossentropy
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [5]:
max_words = 10000
max_len = 100
num_classes = 2

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [6]:
import pandas as pd
data = pd.read_excel("/content/отзывы за лето.xls")

In [7]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [9]:
data = data.dropna()

In [10]:
data.isna().sum()

Rating     0
Content    0
Date       0
dtype: int64

In [11]:
import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

<ipython-input-11-561e11909a58>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = (data['Rating'] > 3)*1


In [12]:
data['target'] = data['target'].astype(int)
data.head()

<ipython-input-12-e937bdd98cf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['target'].astype(int)


,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [13]:
data['target'].value_counts()

1    16721
0     3024
Name: target, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.3, random_state=42, stratify=data['target'])

In [15]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [18]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [19]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [20]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [23]:
x_train

array([[  0,   0,   0, ...,  56, 139, 314],
       [  0,   0,   0, ..., 690, 242, 786],
       [  0,   0,   0, ...,   0,   0, 465],
       ...,
       [  0,   0,   0, ...,   0,   7,   1],
       [  0,   0,   0, ...,   0,   7,   1],
       [  0,   0,   0, ...,   0,  27, 189]], dtype=int32)

In [24]:
x_train.shape, x_test.shape

((13821, 100), (5924, 100))

In [25]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [26]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [27]:
y_train.shape, y_test.shape

((13821, 2), (5924, 2))

#### Слой tf.keras.layers.Embedding по умолчанию:

In [28]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 98, 128)           49280     
                                                                 
 activation (Activation)     (None, 98, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                        

In [30]:
#tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
#early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
#                    callbacks=[tensorboard, early_stopping]
                   )

Epoch 1/5
25/25 [==============================] - 11s 421ms/step - loss: 0.4691 - accuracy: 0.8264 - val_loss: 0.3649 - val_accuracy: 0.8438
Epoch 2/5
25/25 [==============================] - 10s 405ms/step - loss: 0.2735 - accuracy: 0.8767 - val_loss: 0.2272 - val_accuracy: 0.9046
Epoch 3/5
25/25 [==============================] - 10s 382ms/step - loss: 0.1621 - accuracy: 0.9330 - val_loss: 0.2065 - val_accuracy: 0.9147
Epoch 4/5
25/25 [==============================] - 9s 359ms/step - loss: 0.1171 - accuracy: 0.9559 - val_loss: 0.2102 - val_accuracy: 0.9067
Epoch 5/5
25/25 [==============================] - 10s 403ms/step - loss: 0.0867 - accuracy: 0.9713 - val_loss: 0.2183 - val_accuracy: 0.9111


val_accuracy: 0.9111

In [31]:
model.evaluate(x_test, y_test)

186/186 [==============================] - 2s 8ms/step - loss: 0.1909 - accuracy: 0.9227


[0.19088396430015564, 0.9226873517036438]

accuracy: 0.9227

####  Слой tf.keras.layers.Embedding с предобученными векторами:

In [32]:
!pip install gensim

In [33]:
import gensim
from gensim.models import KeyedVectors

In [34]:
!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2023-08-28 16:19:53--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  19.6MB/s    in 20s     

2023-08-28 16:20:13 (16.9 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]

gzip: taiga_upos_skipgram_300_2_2018.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
vocab_size = len(tokenizer.word_index) + 1

In [36]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [37]:
embedding_dim = 300
hits = 0
missed = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        missed += 1

print("Converted %d words (%d missed)." % (hits, missed))

Converted 5381 words (7255 missed)


In [38]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 300)          3791100   
                                                                 
 conv1d_1 (Conv1D)           (None, 98, 128)           115328    
                                                                 
 activation_3 (Activation)   (None, 98, 128)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 activation_4 (Activation)   (None, 10)                0         
                                                      

In [40]:
#tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
#early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
#                    callbacks=[tensorboard, early_stopping]
                   )

Epoch 1/5
25/25 [==============================] - 15s 574ms/step - loss: 0.5484 - accuracy: 0.8302 - val_loss: 0.4261 - val_accuracy: 0.8438
Epoch 2/5
25/25 [==============================] - 17s 672ms/step - loss: 0.3463 - accuracy: 0.8472 - val_loss: 0.2916 - val_accuracy: 0.8438
Epoch 3/5
25/25 [==============================] - 18s 685ms/step - loss: 0.2716 - accuracy: 0.8655 - val_loss: 0.2567 - val_accuracy: 0.8915
Epoch 4/5
25/25 [==============================] - 14s 565ms/step - loss: 0.2393 - accuracy: 0.9019 - val_loss: 0.2437 - val_accuracy: 0.8959
Epoch 5/5
25/25 [==============================] - 14s 567ms/step - loss: 0.2197 - accuracy: 0.9121 - val_loss: 0.2381 - val_accuracy: 0.8952


val_accuracy: 0.8952

In [41]:
model.evaluate(x_test, y_test)

186/186 [==============================] - 4s 20ms/step - loss: 0.2301 - accuracy: 0.9075


[0.23006175458431244, 0.9074949622154236]

accuracy: 0.9075

Качество предсказаний модели с эмбеддингами "из коробки" выше, чем у модели с предобученными векторами.